# Virtual Kinetics Lab Reaction Software - Online Workshop
# OpenMKM Input and Output
This notebook describes pmutt's functionality to read and write OpenMKM CTI files. We will use the NH<sub>3</sub> formation mechanism as a case study.

## Topics Covered
- Read species *ab-initio* data, reactions, lateral interactions, phases and reactor conditions from a spreadsheet
- Write the input thermo (cti and yaml) and reactor (yaml) files that can be read by OpenMKM

## References
- [A Python Multiscale Thermochemistry Toolbox (pMuTT) for thermochemical and kinetic parameter estimation; Lym, Wittreich, et al; Computer Physics Communications, 2020](https://scholar.google.com/scholar?oi=bibs&cluster=17816609206929175595&btnI=1&hl=en)
- [Python Group Additivity (pGrAdd) software for estimating species thermochemical properties; Wittreich, et al; Computer Physics Communications, 2022](https://scholar.google.com/scholar?oi=bibs&cluster=7963801704969122168&btnI=1&hl=en)
- [Microkinetic modeling of surface catalysis; Wittreich, et al; Handbook of Materials Modeling: Applications, 2020](https://scholar.google.com/scholar?oi=bibs&cluster=2412312200916247815&btnI=1&hl=en)

## Input Spreadsheet
All the data will be imported from the [`./inputs/NH3_Input_data.xlsx`](https://github.com/VlachosGroup/pmutt/blob/master/docs/source/examples_jupyter/openmkm_io/inputs/NH3_Input_Data.xlsx) file. There are eight sheets:
1. `refs` contains *ab-initio* and experimental data for a handful of gas species to calculate references
2. `species` contains *ab-initio* data for each specie
3. `reactions` contains elementary steps
4. `phases` contains phases for the species
5. `lateral_interactions` contains lateral interactions between species
6. `beps` contains the Bronsted-Evans-Polyani relationships
7. `reactor` contains description, reaction condition, and solver parameters
8. `units` contains the units of measure for the input data

The contents are displayed below:

**Units**

<img src="images/units.png" width=500>

**References**

<img src="images/refs.png" width=1000>

**Species**

<img src="images/species.png" width=1000>

**Phases**

<img src="images/phases.png" width=1000>

**Reactions**

<img src="images/reactions.png" width=1000>

**Bronsted-Evans-Polyani (BEP) Relationships**

<img src="images/beps.png" width=500>

**Lateral Interactions**

<img src="images/lateral1.png" width=650>

<img src="images/lateral2.png" witdh=300>

**Reactor**

<img src="images/reactor.png" width=1000>

## Designate Units
First, we will designate the units to write the YAML file.

In [1]:
import numpy as np
import os
from pmutt.io.excel import read_excel
from pmutt.omkm.units import Units

file_path = './'
input_filename = 'inputs/NH3_Input_Data.xlsx'
input_path = os.path.join(file_path, input_filename)
output_filename = 'outputs/thermo.yaml'
output_path = os.path.join(file_path, output_filename)

units = Units(**read_excel(io=input_path, sheet_name='units')[0])

## Reading data
Before we can initialize our species, we need the references.

### Reading References
We will open the [input spreadsheet](https://github.com/VlachosGroup/pmutt/blob/master/docs/source/examples_jupyter/openmkm_io/inputs/NH3_Input_Data.xlsx) and read the `refs` sheet.

In [2]:
import os
from pathlib import Path

from pmutt.io.excel import read_excel
from pmutt.empirical.references import Reference, References

# Find the location of Jupyter notebook
# Note that normally Python scripts have a __file__ variable but Jupyter notebook doesn't.
# Using pathlib can overcome this limiation
notebook_path = Path().resolve()
os.chdir(notebook_path)
input_path = './inputs/NH3_Input_Data.xlsx'

refs_data = read_excel(io=input_path, sheet_name='refs')
refs = [Reference(**ref_data) for ref_data in refs_data]
refs = References(references=refs)
print(refs.offset)

{'H': -129.34222830159828, 'N': -320.10077207763874, 'Ru': 0.0}


### Reading Species

In [3]:
from pmutt.empirical.nasa import Nasa

# Lower and higher temperatures
T_low = 298. # K
T_high = 800. # K

species_data = read_excel(io=input_path, sheet_name='species')
species = []
species_phases = {}
for ind_species_data in species_data:
    # Initialize NASA from statistical mechanical data
    ind_species = Nasa.from_model(T_low=T_low, T_high=T_high, references=refs, **ind_species_data)
    species.append(ind_species)

    # Group the species by phase for later use
    try:
        species_phases[ind_species.phase].append(ind_species)
    except KeyError:
        species_phases[ind_species.phase] = [ind_species]

### Adding species from other empirical sources

In [4]:
import numpy as np
from pmutt.empirical.shomate import Shomate

Ar = Shomate(name='Ar', elements={'Ar': 1}, phase='gas', T_low=298., T_high=6000.,
             a=np.array([20.78600, 2.825911e-7, -1.464191e-7, 1.092131e-8, -3.661371e-8, -6.19735, 179.999, 0.]))

species.append(Ar)
species_phases['gas'].append(Ar)

### Reading BEP

In [5]:
from pmutt.omkm.reaction import BEP

beps_data = read_excel(io=input_path, sheet_name='beps')
beps = []
for bep_data in beps_data:
    beps.append(BEP(**bep_data))

# Combine species and BEPs to make reactions
species_with_beps = species + beps

### Read reactions

In [6]:
from pmutt import pmutt_list_to_dict
from pmutt.omkm.reaction import SurfaceReaction

# Convert species to dictionary for easier reaction assignment
species_with_beps_dict = pmutt_list_to_dict(species_with_beps)
reactions_data = read_excel(io=input_path, sheet_name='reactions')
reactions = []
# Store information about phases for later retrieval
reaction_phases = {}
for reaction_data in reactions_data:
    reaction = SurfaceReaction.from_string(species=species_with_beps_dict,
                                           **reaction_data)
    reactions.append(reaction)
    # Assign phase information
    reaction_species = reaction.get_species(include_TS=True)
    for ind_species in reaction_species:
        try:
            phase = species_with_beps_dict[ind_species].phase
        except AttributeError:
            pass
        # Assign if key already exists
        if phase in reaction_phases:
            if reaction not in reaction_phases[phase]:
                reaction_phases[phase].append(reaction)
        else:
            reaction_phases[phase] = [reaction]

### Read lateral interactions

In [7]:
from pmutt.mixture.cov import PiecewiseCovEffect

interactions = []
interactions_data = read_excel(io=input_path, sheet_name='lateral_interactions')
interaction_phases = {}
for interaction_data in interactions_data:
    interaction = PiecewiseCovEffect(**interaction_data)
    interactions.append(interaction)

    # Assign phase information
    phase = species_with_beps_dict[interaction.name_i].phase
    # Assign if key already exists
    if phase in interaction_phases:
        if interaction not in interaction_phases[phase]:
            interaction_phases[phase].append(interaction)
    else:
        interaction_phases[phase] = [interaction]

### Reading Phases

In [8]:
from pmutt.omkm.phase import IdealGas, InteractingInterface, StoichSolid

phases_data = read_excel(io=input_path, sheet_name='phases')
phases = []
for phase_data in phases_data:
    # Pre-processing relevant data
    phase_name = phase_data['name']
    phase_type = phase_data.pop('phase_type')
    phase_data['species'] = species_phases[phase_name]

    # Create the appropriate object
    if phase_type == 'IdealGas':
        phase = IdealGas(**phase_data)
    elif phase_type == 'StoichSolid':
        phase = StoichSolid(**phase_data)
    elif phase_type == 'InteractingInterface':
        phase_data['reactions'] = reaction_phases[phase_name]
        phase_data['interactions'] = interaction_phases[phase_name]
        phase = InteractingInterface(**phase_data)
    phases.append(phase)

## Output Thermo YAML File
If you would prefer to return the file as a string instead of writing it, omit the ``filename``.

In [9]:
from pmutt.io.omkm import write_thermo_yaml

output_path = './outputs/thermo.yaml'
use_motz_wise = True
write_thermo_yaml(reactions=reactions, species=species, phases=phases, units=units,
                  lateral_interactions=interactions, filename=output_path,
                  use_motz_wise=use_motz_wise)

In [10]:
print(write_thermo_yaml(reactions=reactions, species=species, phases=phases, units=units,
                        lateral_interactions=interactions, use_motz_wise=use_motz_wise))

# File generated by pMuTT (v 1.4.9) on 2023-12-11 15:42:16.763382
# See documentation for OpenMKM YAML file here:
# https://vlachosgroup.github.io/openmkm/input

#-------------------------------------------------------------------------------
# UNITS
#-------------------------------------------------------------------------------
units: {mass: g, length: cm, time: s, quantity: mol, energy: kcal, activation-energy: kcal/mol,
  pressure: atm}


#-------------------------------------------------------------------------------
# PHASES
#-------------------------------------------------------------------------------
phases:

- name: gas
  elements: [Ar, N, H]
  species: [N2, NH3, H2, Ar]
  thermo: ideal-gas
  kinetics: gas
  reactions: none

- name: bulk
  elements: [Ru]
  species: [RU(B)]
  thermo: ref-state-fixed-stoichiometry

- name: terrace
  elements: [Ru, N, H]
  species: [N2(T), N(T), H(T), NH3(T), NH2(T), NH(T), RU(T)]
  kinetics: surface
  site-density: "2.1671e-09 mol/cm^2"
  ther

## Output Reactor YAML File
If you would prefer to return the file as a string instead of writing it, omit the ``filename``.

In [11]:
from pmutt.io.omkm import write_yaml

Path('./outputs').mkdir(exist_ok=True)
yaml_path = './outputs/reactor.yaml'
reactor_data = read_excel(io=input_path, sheet_name='reactor')[0]
write_yaml(filename=yaml_path, phases=phases, units=units, **reactor_data)
print('Reactor YAML\n')
print(write_yaml(phases=phases, units=units, **reactor_data))

Reactor YAML

# File generated by pMuTT (v 1.4.9) on 2023-12-11 15:42:16.904037
# See documentation for OpenMKM YAML file here:
# https://vlachosgroup.github.io/openmkm/input
inlet_gas:
    flow_rate: "1 cm3/s"
phases:
    bulk:
        name: bulk
    gas:
        initial_state: "NH3:1.0"
        name: gas
    surfaces:
    -   initial_state: "RU(T):1.0"
        name: terrace
    -   initial_state: "RU(S):1.0"
        name: step
reactor:
    cat_abyv: "1500 /cm"
    pressure: "1 atm"
    temperature: 900
    temperature_mode: "isothermal"
    type: "cstr"
    volume: "1 cm3"
simulation:
    end_time: "5 s"
    init_step: 1.0e-15
    output_format: "csv"
    solver:
        atol: 1.0e-15
        rtol: 1.0e-10
    stepping: "logarithmic"
    transient: true

